In [ ]:
"""
(Topic modeling continued)
Use Tf-idf and NMF

"""

## 4. Topic modeling - NMF

Besides LDA, there are other matrix factorization techniques such as Latent Semantic Indexing (**LSI**) and non-negative Matrix Factorization (**NMF**).

NMF is similar to Principal component analysis (**PCA**). Vectors are non-negative; by factoring them into the lower-dimensional form, coefficients are also non-negative.